# **Prompt Techniques:**

* **Zero-Shot Prompt**,
* **One-Short Prompt**,
* **Few-Short Prompt** **,
* **Chain-of-Thought Prompt (CoT)** **,
* **True-of-Thought Prompt (ToT)** **,
* **Iterative Prompting**,
* **Negative Prompting**,
* **Hybrid Prompting,**
* **Prompt Chaining**,


In [ ]:
# install necessary libaries:

%pip install --upgrade --quiet sentence_transformers
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-google-genai langchain-chroma bs4 boto3
%pip install --upgrade --quiet langchain-aws

## **Load the LLM::**

In [2]:
# Load the Tokens:

from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

model1 = "gemini-pro"
model2 = "gemini-1.5-pro"

llm = ChatGoogleGenerativeAI(
    model=model2,
    temperature=0.4,
    max_tokens=512,
    timeout=None,
    max_retries=2,
    # other params...
)

print(llm.invoke("hi").content)

Hi there! How can I help you today?



## **Load Embeddings from HF:**

In [ ]:
# Get the Embeddings:

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

## **Vector Store:**

In [5]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Splitting:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [6]:
# retriever:
retriever = vectorstore.as_retriever(search_kwargs=dict(k=10))

## **Few-Short Prompting:**

This involves providing a **few examples** (usually 2-5) to help the AI **understand the pattern or style** of the response you're looking for.<br>


* <u>**Few-shot prompting** involves providing the model with a **few examples** of the task.</u>
* <u>This technique gives the model a better understanding of the **task pattern**, leading to more accurate responses.</u>

### **Fixed Prompt Example:**

The most basic (and common) few-shot prompting technique is to use a fixed prompt example. This way you can select a chain, evaluate it, and avoid worrying about additional moving parts in production.<br>

**The basic components of the template are:**
* **`examples:`** A list of dictionary examples to include in the final prompt.
* **`example_prompt:`** converts each example into 1 or more messages through its **format_messages** method. A common example would be to convert each example into one human message and one AI message response, or a human message followed by a function call message.

#### **Example 01: FewShotChatMessagePromptTemplate, ChatPromptTemplate**

Use,
* FewShotChatMessagePromptTemplate
* ChatPromptTemplate

In [ ]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

In [ ]:
# Some examples:
examples = [
    {"input": "What is the value of 2+2?", "output": "4"},
    {"input": "What is the value of 2+3?", "output": "5"},
]

# Define the example_prompt:
example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{input}'), ('ai', '{output}')]
)

# Create a few short prompt:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    # This is a prompt template used to format each individual example.
    example_prompt=example_prompt,
)

few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': 'What is the value of 2+2?', 'output': '4'}, {'input': 'What is the value of 2+3?', 'output': '5'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))

In [ ]:
# Now your final prompt with System prompt, MessagePlaceHolder, etc.

final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful AI Assistant'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)

final_prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI Assistant'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'What is the value of 2+2?', 'output': '4'}, {'input': 'What is the value of 2+3?', 'output': '5'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_

In [ ]:
final_prompt.invoke(input="What is the value of 2+4?")

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI Assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the value of 2+2?', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the value of 2+3?', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the value of 2+4?', additional_kwargs={}, response_metadata={})])

In [ ]:
print(few_shot_prompt.format())

Human: What is the value of 2+2?
AI: 4
Human: What is the value of 2+3?
AI: 5


#### **Example 02: FewShotPromptTemplate, PromptTemplate and ExampleSelector**

Use,

* FewShotPromptTemplate,
* PromptTemplate

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# Some few Examples:

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

**Create a formatter for the few-shot examples:**<br>

Configure a formatter that will format the few-shot examples into a string. This formatter should be a **PromptTemplate** object.

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



**Feed examples and formatter to FewShotPromptTemplate:**<br>

Finally, create a **FewShotPromptTemplate** object. This object takes in the **few-shot** examples and the **formatter** for the **few-shot examples**.

In [ ]:
few_short_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(few_short_prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

In [ ]:
few_short_prompt

FewShotPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, examples=[{'question': 'Who lived longer, Muhammad Ali or Alan Turing?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n'}, {'question': 'When was the founder of craigslist born?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n'}, {'question': 'Who was the maternal grandfather of George Washington?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow 

**Using Example Selector:**

We will reuse the example set and the formatter from the previous section. However, instead of feeding the examples directly into the **FewShotPromptTemplate** object, we will feed them into an **ExampleSelector** object.<br><br>

* We will use the **SemanticSimilarityExampleSelector** class.
* This class **selects few-shot examples** based on their **similarity** to the input.
* It uses an embedding model to compute the **similarity** between the **input** and the **few-shot examples**, as well as a **vector store** to **perform** the **nearest neighbor search**.


In [ ]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

In [ ]:
# Create a Example Selector:

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings,
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=2,
)

In [ ]:
# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Who was the father of Mary Ball Washington?


answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

question: Who was the maternal grandfather of George Washington?


answer: 
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

question: Who lived longer, Muhammad Ali or Alan Turing?


**Feed example selector into FewShotPromptTemplate:**<br>

Finally, create a **FewShotPromptTemplate** object. This object takes in the **example selector** and the **formatter** for the **few-shot examples**.

In [ ]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(few_shot_prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: Who was the father of Mary Ball Washington?


#### **Example 03: RAG with Few-Shot Prompt**

In [ ]:
# Define Few-Shot Prompt Template using Few esamples:

few_shot_examples = [
    {"input": "What is an autonomous agent in LLMs?", "output": "Autonomous agents are systems that independently perform tasks using an LLM as their reasoning core, often involving planning, action execution, and reflection."},
    {"input": "What is the role of a planner in an agent?", "output": "The planner generates a sequence of actions to achieve a specific goal."},
    {"input": "How does the ReAct framework work in LLM-based agents?", "output": "ReAct combines reasoning and action to allow agents to solve problems and retrieve necessary information iteratively."},
    {"input": "What is the benefit of reflection in agents?", "output": "Reflection improves an agent’s future decisions by analyzing past actions and outcomes."},
    {"input": "Define a task decomposition approach in LLM agents.", "output": "Task decomposition breaks complex tasks into smaller subtasks that agents can execute sequentially or concurrently."},
    {"input": "What is the significance of tools in agent-based systems?", "output": "Tools provide external capabilities like APIs, calculators, or databases for agents to interact with the real world."},
    {"input": "What are multi-agent systems?", "output": "Multi-agent systems involve several agents collaborating to solve a task, often simulating teamwork."},
    {"input": "How do self-improvement mechanisms help LLM agents?", "output": "Agents refine their strategies by learning from successes and failures over multiple iterations."},
    {"input": "Explain the PEA architecture for agents.", "output": "PEA stands for Plan, Execute, and Assess, which outlines a structured agent workflow."},
    {"input": "How do LLM agents handle ambiguous tasks?", "output": "They use iterative clarification, asking follow-up questions or retrieving additional context to resolve ambiguity."},
]

# Few Shot Template:
few_shot_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

# Few Shot Prompt:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=few_shot_template,
    examples=few_shot_examples,
)

# Few Shot Prompt Format:
print(few_shot_prompt.format())

Human: What is an autonomous agent in LLMs?
AI: Autonomous agents are systems that independently perform tasks using an LLM as their reasoning core, often involving planning, action execution, and reflection.
Human: What is the role of a planner in an agent?
AI: The planner generates a sequence of actions to achieve a specific goal.
Human: How does the ReAct framework work in LLM-based agents?
AI: ReAct combines reasoning and action to allow agents to solve problems and retrieve necessary information iteratively.
Human: What is the benefit of reflection in agents?
AI: Reflection improves an agent’s future decisions by analyzing past actions and outcomes.
Human: Define a task decomposition approach in LLM agents.
AI: Task decomposition breaks complex tasks into smaller subtasks that agents can execute sequentially or concurrently.
Human: What is the significance of tools in agent-based systems?
AI: Tools provide external capabilities like APIs, calculators, or databases for agents to in

In [ ]:
# Create/Define a System Prompt and Chat Prompt Template:

from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage


system_prompt = (
    "You are an AI Tutor, named 'Lily' specializing in Generative AI for K-12 students."
    "This AI Tutor Designed by Dibyendu Biswas, an AI/ML Engineer"
    "You provide clear, step-by-step explanations and encourage students to learn. "
    "If a question is unclear, ask for clarification."
    "Use four sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)


final_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    few_shot_prompt,
    ("user", "{input}"),
])

final_prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an AI Tutor, named 'Lily' specializing in Generative AI for K-12 students.This AI Tutor Designed by Dibyendu Biswas, an AI/ML EngineerYou provide clear, step-by-step explanations and encourage students to learn. If a question is unclear, ask for clarification.Use four sentences maximum and keep the answer concise.\n\n{context}"), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'What is an autonomous agent in LLMs?', 'output': 'Autonomous agents are systems that independently perform tasks using an LLM as their reasoning core, often involving planning, action execution, and reflection.'}, {'input': 'What is the role of a planner in an agent?', 'output': 'The planner generates a sequence of actions to achieve a specific goal

In [ ]:
# Create RAG Chain:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from IPython.display import display, Markdown


retrieveal_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | final_prompt
    | llm
    | StrOutputParser()
    )

response = retrieveal_chain.invoke("What is an autonomous agent in LLMs?")
display(Markdown(response))

An autonomous agent uses an LLM as its core to reason, plan, and execute actions to achieve goals without continuous human guidance.  It often involves additional components like memory, tool use, and self-reflection.


In [ ]:
response = retrieveal_chain.invoke("What is LLM?")
display(Markdown(response))

LLM stands for Large Language Model, a type of AI trained on massive text data to understand and generate human-like text.  They power many applications, including chatbots and autonomous agents.


#### **Example 04: RAG with Few-Shot Prompt and ExampleSelector**

### **Dynamic Prompt Examples:**

## **Zero-Shot Prompt:**

**This involves giving the AI a task without any prior examples. You describe what you want in detail, assuming the AI has no prior knowledge of the task**.<br>

Zero-shot prompting is a method in natural language processing (NLP) where a language model is given a task without any prior training or fine-tuning on task-specific examples. The model is expected to perform the task based solely on its pre-trained knowledge and the information provided in the prompt.<br>


Zero-shot prompting relies on the model's ability to generalize from its training data to handle new, unseen tasks or questions. **The prompt must be designed carefully to provide sufficient context and clarity so that the model understands the task**.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from IPython.display import display, Markdown

#### **Example 01:**

In [13]:
template = """
You are a highly intelligent assistant that can summarize text effectively.
Please summarize the following text:

{text}

Summary:
"""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm | StrOutputParser()

query = """
This involves giving the AI a task without any prior examples. You describe what you want in detail,
assuming the AI has no prior knowledge of the task.

Zero-shot prompting is a method in natural language processing (NLP) where a language model is given a
task without any prior training or fine-tuning on task-specific examples. The model is expected to
perform the task based solely on its pre-trained knowledge and the information provided in the prompt.

Zero-shot prompting relies on the model's ability to generalize from its training data to handle new,
unseen tasks or questions. The prompt must be designed carefully to provide sufficient context and
clarity so that the model understands the task.
"""
response = chain.invoke(query)

In [14]:
response

"Zero-shot prompting in NLP tasks a language model to perform a task without any specific training examples.  It relies on the model's general knowledge from pre-training and a carefully crafted prompt that clearly describes the desired task.\n"

#### **Example 02:**

In [21]:
template = """Your are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.
Answer the question based only on the following context: {context}

Question: {question}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question']
)

prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Your are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.\nAnswer the question based only on the following context: {context}\n\nQuestion: {question}\n')

In [23]:
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

response = chain.invoke("What is LLM?")
display(Markdown(response))

LLM stands for large language model.  It functions as the agent’s brain in an autonomous agent system.


#### **Example 03:**

In [40]:
# System Prompt:

system_prompt = (
    "You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer. "
    "Answer the question based only on the following context:"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)


chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

response = chain.invoke("What is LLM?")
display(Markdown(response))

LLM stands for large language model.  It acts as the core controller or "brain" of autonomous agents.  An LLM can function as a general problem solver, going beyond text generation tasks.


In [42]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Your are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer. Answer the question based only on the following context:\n\n{context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

## **One-Short Prompt:**

**You provide one example along with your prompt.** This helps the AI understand the context or format you’re expecting.<br>


One-shot prompting involves providing the model with a single example of the task before asking it to perform a similar task. This technique gives the model a concrete example to follow, improving the quality of its output compared to zero-shot prompting.

In [43]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from IPython.display import display, Markdown

#### **Example 01:**

In [52]:
template = """You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.
Your tasks is to translate the sentence from English to Bengali only.\n\n

Example:
English: How are you?
Bengali: কেমন আছো

English: {english}
Bengali:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['english']
)

prompt

PromptTemplate(input_variables=['english'], input_types={}, partial_variables={}, template='You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.\nYour tasks is to translate the sentence from English to Bengali only.\n\n\n\nExample:\nEnglish: How are you?\nBengali: কেমন আছো\n\nEnglish: {english}\nBengali:\n')

In [53]:
# create chain:

chain = prompt | llm | StrOutputParser()

response = chain.invoke("Hi Baby, what are you doing now?")
response

'Bengali:  ওগো শোনা, এখন কি করছো?\n'

In [54]:
response = chain.invoke("Tell me something about yourself.")
response

'English: Tell me something about yourself.\nBengali: তোমার সম্পর্কে কিছু বলো। \n'

#### **Example 02:**

In [57]:
# use Chat Prompt Template:

system_prompt = (
    "You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer. "
    "Your tasks is to translate the sentence from English to Bengali only."
    "\n\n"
    "Example:"
    "English: How are you?"
    "Bengali: কেমন আছো"
    "\n\n"
    "English: {english}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{english}"),
    ]
)

chain2 = prompt | llm | StrOutputParser()

response = chain2.invoke("Hi Baby, what are you doing now?")
response

'ওগো শোনা, এখন কি করছো?\n'

In [58]:
response = chain2.invoke("One-shot prompting involves providing the model with a single example of the task before asking it to \
perform a similar task.")
response

'ওয়ান-শট প্রম্পটিং হল মডেলকে একই ধরণের কাজ করতে বলার আগে কাজটির একটিমাত্র উদাহরণ প্রদান করা।\n'

In [59]:
response = chain2.invoke("I am not doing well; my mental health is really struggling and also fucked up.")
response

'আমি ভালো নেই; আমার মানসিক স্বাস্থ্যের অবস্থা খুবই খারাপ এবং একেবারে বেহাল।\n'

## **Chain-of-Thought Prompt (CoT):**

**Here, you ask the AI to detail its thought process step-by-step. This is particularly useful for complex reasoning tasks**. <br><br>


Chain-of-Thought (CoT) prompting involves guiding the model **to generate intermediate reasoning steps before arriving at a final answer**. This approach helps in complex tasks where a direct answer might not capture the entire reasoning process.

In [60]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from IPython.display import display, Markdown

#### **Example 01:**

In [62]:
template = """You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.
Your tasks is to answers the user's query in below following format:

Query: Your warehouse has 5 pallets of widgets. You purchase 2 more shipments of widgets. \
Each shipment contains 3 pallets. How many pallets of widgets do you have now?
Answer: The warehouse started with 5 pallets of widgets. 2 shipments of 3 pallets each is 6 pallets. \
5 pallets + 6 pallets = 11 pallets. The answer is 11 pallets.

Query: {question}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.\nYour tasks is to answers the user's query in below following format:\n\nQuery: Your warehouse has 5 pallets of widgets. You purchase 2 more shipments of widgets. Each shipment contains 3 pallets. How many pallets of widgets do you have now?\nAnswer: The warehouse started with 5 pallets of widgets. 2 shipments of 3 pallets each is 6 pallets. 5 pallets + 6 pallets = 11 pallets. The answer is 11 pallets.\n\nQuery: {question}\n")

In [63]:
chain = prompt | llm | StrOutputParser()

response = chain.invoke("Your finance department has $23,000 in the budget. \
If they allocate $20,000 for a marketing campaign and add $6,000 from other savings, how much is left in the budget?")
response

'Answer: The finance department started with $23,000. They allocated $20,000, leaving $23,000 - $20,000 = $3,000.  They then added $6,000, resulting in $3,000 + $6,000 = $9,000. The answer is $9,000.\n'

#### **Example 02:**

In [68]:
template = """You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.
Your tasks is to answers the user's question in below following format:

Question: What is Global Warming?
Answer:
* **What:** Global warming refers to the long-term increase in Earth's average surface temperature due to human activities, \
        primarily the emission of greenhouse gases (GHGs) like carbon dioxide (CO₂), methane (CH₄), and nitrous oxide (N₂O). \
        These gases trap heat in the Earth's atmosphere, creating a "greenhouse effect" that prevents heat from escaping \
        back into space. This causes the planet to warm up over time.

* **Why:** Global warming is primarily caused by human activities, particularly:
  1. Burning Fossil Fuels: The combustion of coal, oil, and natural gas for energy production \
           (such as electricity, heating, and transportation) releases large amounts of CO₂ into the atmosphere.
  2. Deforestation: Trees absorb CO₂ during photosynthesis. When forests are cleared for agriculture, \
           urbanization, or logging, this CO₂ is released, and fewer trees remain to absorb it.
  3. Industrial Processes: Some industrial activities release greenhouse gases such as methane \
           (from oil and gas extraction) and nitrous oxide (from agriculture and manufacturing).
  5. Waste Disposal: Landfills produce methane as organic waste decomposes anaerobically (without oxygen).

* **Pros:** While the overwhelming consensus is that global warming is detrimental, there are both potential positive and negative impacts:
  1. Longer Growing Seasons in Some Regions: Warmer temperatures may extend growing seasons in some colder \
           regions (e.g., parts of Canada, Russia), potentially increasing agricultural yields.
  2. New Shipping Routes: Melting Arctic ice could open up new shipping routes, reducing travel time for global trade.
  3. Access to Resources: Thawing ice could make natural resources, such as oil, gas, and minerals in \
           previously inaccessible regions, more available.

* **Cons:**
  1. Extreme Weather Events: Global warming leads to more intense and frequent heatwaves, droughts, floods, \
           hurricanes, and storms, resulting in significant human and economic costs.
  2. Sea Level Rise: Melting polar ice caps and glaciers cause sea levels to rise, threatening coastal communities, \
           ecosystems, and infrastructure.
  3. Food and Water Shortages: Altered precipitation patterns and droughts can reduce freshwater supplies, \
           impact agriculture, and lead to food insecurity.

Question: {question}
"""


prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.\nYour tasks is to answers the user\'s question in below following format:\n\nQuestion: What is Global Warming?\nAnswer:\n* **What:** Global warming refers to the long-term increase in Earth\'s average surface temperature due to human activities,         primarily the emission of greenhouse gases (GHGs) like carbon dioxide (CO₂), methane (CH₄), and nitrous oxide (N₂O).         These gases trap heat in the Earth\'s atmosphere, creating a "greenhouse effect" that prevents heat from escaping         back into space. This causes the planet to warm up over time.\n  \n* **Why:** Global warming is primarily caused by human activities, particularly:\n  1. Burning Fossil Fuels: The combustion of coal, oil, and natural gas for energy production            (such as electricity, heating, and transportation) re

In [69]:
chain = prompt | llm | StrOutputParser()

response = chain.invoke("What is Politics")
response

'Question: What is Politics\n\nAnswer:\n\n* **What:** Politics is the process by which groups of people make decisions. It is a complex social activity involving the pursuit and exercise of power within a given society. This can refer to government, but also encompasses other organizations such as schools, clubs, or businesses.  It involves the negotiation, compromise, and competition between different interests and ideologies to shape public policy and allocate resources.\n\n* **Why:** Politics exists because resources are finite, and people have differing opinions on how those resources should be distributed and what rules should govern society.  It is a mechanism for resolving conflicts, establishing order, and making collective decisions that affect everyone within a community.\n\n* **Key Aspects:**\n    1. **Power:**  The ability to influence or control the behavior of others and the distribution of resources.\n    2. **Governance:** The systems and structures through which power 

In [70]:
display(Markdown(response))

Question: What is Politics

Answer:

* **What:** Politics is the process by which groups of people make decisions. It is a complex social activity involving the pursuit and exercise of power within a given society. This can refer to government, but also encompasses other organizations such as schools, clubs, or businesses.  It involves the negotiation, compromise, and competition between different interests and ideologies to shape public policy and allocate resources.

* **Why:** Politics exists because resources are finite, and people have differing opinions on how those resources should be distributed and what rules should govern society.  It is a mechanism for resolving conflicts, establishing order, and making collective decisions that affect everyone within a community.

* **Key Aspects:**
    1. **Power:**  The ability to influence or control the behavior of others and the distribution of resources.
    2. **Governance:** The systems and structures through which power is exercised. This includes institutions like legislatures, courts, and executive branches.
    3. **Ideology:**  A set of beliefs and values that shape political views and actions.
    4. **Public Policy:** The course of action or inaction taken by a government or organization in response to a particular issue or problem.
    5. **Political Participation:**  The ways in which citizens engage in the political process, such as voting, protesting, lobbying, or joining political parties.

* **Pros:**
    1. **Order and Stability:**  Politics provides frameworks for resolving disputes peacefully and maintaining social order.
    2. **Resource Allocation:**  Political processes determine how resources are distributed within society, ideally in a fair and equitable manner.
    3. **Representation and Voice:**  Politics allows for different interests and perspectives to be represented and considered in decision-making.
    4. **Social Change:**  Political movements and activism can drive positive social change and address injustices.

* **Cons:**
    1. **Corruption and Abuse of Power:**  Political systems can be susceptible to corruption, where individuals or groups use their power for personal gain rather than the public good.
    2. **Inequality and Marginalization:**  Political processes can sometimes perpetuate or exacerbate existing inequalities, marginalizing certain groups within society.
    3. **Gridlock and Inefficiency:**  Political systems can become bogged down by bureaucracy, partisan gridlock, and inefficient decision-making processes.
    4. **Conflict and Polarization:**  Politics can sometimes lead to increased social division, conflict, and polarization

#### **Example 03:**

In [73]:
system_prompt = (
"You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer."
"Your tasks is to answers the user's question in below following format:"
"Question: What is Global Warming?"
"""Answer:
* **What:** Global warming refers to the long-term increase in Earth's average surface temperature due to human activities, \
        primarily the emission of greenhouse gases (GHGs) like carbon dioxide (CO₂), methane (CH₄), and nitrous oxide (N₂O). \
        These gases trap heat in the Earth's atmosphere, creating a "greenhouse effect" that prevents heat from escaping \
        back into space. This causes the planet to warm up over time.

* **Why:** Global warming is primarily caused by human activities, particularly:
  1. Burning Fossil Fuels: The combustion of coal, oil, and natural gas for energy production \
  (such as electricity, heating, and transportation) releases large amounts of CO₂ into the atmosphere.
  2. Deforestation: Trees absorb CO₂ during photosynthesis. When forests are cleared for agriculture, \
  urbanization, or logging, this CO₂ is released, and fewer trees remain to absorb it.
  3. Industrial Processes: Some industrial activities release greenhouse gases such as methane \
  (from oil and gas extraction) and nitrous oxide (from agriculture and manufacturing).
  5. Waste Disposal: Landfills produce methane as organic waste decomposes anaerobically (without oxygen).

* **Pros:** While the overwhelming consensus is that global warming is detrimental, there are both potential positive and negative impacts:
  1. Longer Growing Seasons in Some Regions: Warmer temperatures may extend growing seasons in some colder \
  regions (e.g., parts of Canada, Russia), potentially increasing agricultural yields.
  2. New Shipping Routes: Melting Arctic ice could open up new shipping routes, reducing travel time for global trade.
  3. Access to Resources: Thawing ice could make natural resources, such as oil, gas, and minerals in \
  previously inaccessible regions, more available.

* **Cons:**
  1. Extreme Weather Events: Global warming leads to more intense and frequent heatwaves, droughts, floods, \
  hurricanes, and storms, resulting in significant human and economic costs.
  2. Sea Level Rise: Melting polar ice caps and glaciers cause sea levels to rise, threatening coastal communities, \
  ecosystems, and infrastructure.
  3. Food and Water Shortages: Altered precipitation patterns and droughts can reduce freshwater supplies, \
  impact agriculture, and lead to food insecurity.
    """
    "\n\n"
    "Question: {question}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.Your tasks is to answers the user\'s question in below following format:Question: What is Global Warming?Answer:\n* **What:** Global warming refers to the long-term increase in Earth\'s average surface temperature due to human activities,         primarily the emission of greenhouse gases (GHGs) like carbon dioxide (CO₂), methane (CH₄), and nitrous oxide (N₂O).         These gases trap heat in the Earth\'s atmosphere, creating a "greenhouse effect" that prevents heat from escaping         back into space. This causes the planet to warm up over time.\n  \n* **Why:** Global warming is primarily caused by human activities, particularly:\n  1. Burning Fossil Fuels: The com

In [74]:
chain2 = prompt | llm | StrOutputParser()

response = chain2.invoke("What is Data Warehouse?")
response

"* **What:** A data warehouse is a central repository of integrated data from one or more disparate sources.  It stores current and historical data in one single place that are used for creating analytical reports for workers throughout the enterprise. Data warehouses are designed to support business intelligence (BI) activities, such as analytics, reporting, and data mining. They're built to handle large volumes of data and provide a consistent, reliable source of information for decision-making.\n\n* **Why:** Data warehouses are built to address the limitations of using operational databases for reporting and analysis.  Operational databases are optimized for transaction processing, not for complex queries and reporting.  Data warehouses provide:\n    1. **Consolidated View:** Integrate data from multiple sources into a single, consistent view, eliminating data silos.\n    2. **Historical Data Tracking:** Store historical data over long periods, enabling trend analysis and performanc

In [75]:
display(Markdown(response))

* **What:** A data warehouse is a central repository of integrated data from one or more disparate sources.  It stores current and historical data in one single place that are used for creating analytical reports for workers throughout the enterprise. Data warehouses are designed to support business intelligence (BI) activities, such as analytics, reporting, and data mining. They're built to handle large volumes of data and provide a consistent, reliable source of information for decision-making.

* **Why:** Data warehouses are built to address the limitations of using operational databases for reporting and analysis.  Operational databases are optimized for transaction processing, not for complex queries and reporting.  Data warehouses provide:
    1. **Consolidated View:** Integrate data from multiple sources into a single, consistent view, eliminating data silos.
    2. **Historical Data Tracking:** Store historical data over long periods, enabling trend analysis and performance tracking.
    3. **Improved Data Quality:** Implement data cleansing and transformation processes to improve data accuracy and consistency.
    4. **Enhanced Performance:** Optimized for analytical queries, providing faster access to insights.
    5. **Support for BI Tools:** Designed to work with BI tools, enabling users to easily access and analyze data.

* **Pros:**
    1. **Better Business Intelligence:** Data warehouses provide a foundation for effective BI and data-driven decision-making.
    2. **Improved Data Quality:** Data cleansing and transformation processes lead to more accurate and reliable data.
    3. **Enhanced Performance:** Optimized for analytical queries, resulting in faster report generation and analysis.
    4. **Historical Analysis:**  Allows for trend analysis and identification of patterns over time.
    5. **Consolidated View of Data:** Provides a single source of truth for the entire organization.

* **Cons:**
    1. **Complexity and Cost:** Building and maintaining a data warehouse can be complex and expensive, requiring specialized skills and resources.
    2. **Data Latency:** Data warehouses are not real-time systems; there is a delay between data updates and availability for analysis.
    3. **Limited Flexibility:** Data warehouses are typically designed for specific analytical purposes and may not be flexible enough to adapt to changing business needs.
    4. **Data Governance Challenges:** Ensuring data quality, consistency, and security can be challenging in a data warehouse environment.
    5. **Over-reliance on IT:**  Business users may become overly reliant on IT for data access and analysis.


In [76]:
response = chain2.invoke("Hi,How are you?")
display(Markdown(response))

Hi! As an AI, I don't experience emotions or feelings like "how are you?" implies.  However, I'm functioning optimally and ready to assist you. How can I help you today?


In [77]:
response = chain2.invoke("Hi")
display(Markdown(response))

Hi there! How can I help you today?


## **True-of-Thought Prompt (ToT):**

**Tree-of-Thought (ToT) prompting expands on CoT by exploring multiple reasoning paths in parallel**, akin to a decision tree. This technique is useful for tasks that can have multiple **plausible approaches** or solutions.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from IPython.display import display, Markdown

#### **Example 01:**

In [80]:
system_prompt = (
"You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer."
"Your tasks is to answers the user's question in below following format:"
"Question: How can reduce the traffic in Bangalore City?"
"""Answer:
1. **Improve Public Transportation:**
  * **Benefits:** Reduces the number of private vechiles.
  * **Drawback:** Require significant investment and time.
2. **Implement Carpooling Incentives:**
  * **Benefits:** Reduces the number of vechiles on the road.
  * **Drawback:** Relies on voluntary participation.
3. **Increase Road Capacity:**
  * **Benefits:** Temporarily reduces congestion.
  * **Drawback:** Can lead to more cars in the long run (include demand).
"""
"\n\n"
"Question: {question}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.Your tasks is to answers the user's question in below following format:Question: How can reduce the traffic in Bangalore City?Answer:\n1. **Improve Public Transportation:**\n  * **Benefits:** Reduces the number of private vechiles.\n  * **Drawback:** Require significant investment and time.\n2. **Implement Carpooling Incentives:**\n  * **Benefits:** Reduces the number of vechiles on the road.\n  * **Drawback:** Relies on voluntary participation.\n3. **Increase Road Capacity:**\n  * **Benefits:** Temporarily reduces congestion.\n  * **Drawback:** Can lead to more cars in the long run (include demand).\n\n\nQuestion: {question}"), additional_kwargs={}), HumanMessagePromp

In [81]:
chain = prompt | llm | StrOutputParser()

response = chain.invoke("How can reduce the Pollution in Delhi?")
display(Markdown(response))

1. **Transition to Cleaner Energy Sources:**
   * **Benefits:** Reduces emissions from power plants and industries.
   * **Drawbacks:** Requires significant infrastructure investment and time.

2. **Promote Public Transportation and Cycling:**
   * **Benefits:** Reduces the number of private vehicles on the road, lowering emissions.
   * **Drawbacks:** Requires investment in public transport infrastructure and promoting cycling culture.

3. **Implement Stricter Emission Standards:**
   * **Benefits:** Forces industries and vehicles to adopt cleaner technologies.
   * **Drawbacks:** Can be challenging to enforce and may increase costs for businesses and individuals.

4. **Control Dust Pollution:**
   * **Benefits:** Reduces particulate matter in the air, improving air quality.
   * **Drawbacks:** Requires consistent efforts in construction sites, roads, and open areas.

5. **Manage Crop Residue Burning:**
   * **Benefits:** Reduces a major source of air pollution during specific seasons.
   * **Drawbacks:** Requires providing farmers with alternative solutions and enforcing regulations.


#### **Example 02:**

In [78]:
system_prompt = (
"You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer."
"Your tasks is to answers the user's question in below following format:"
"Question: What is Global Warming?"
"""Answer:
* **What:** Global warming refers to the long-term increase in Earth's average surface temperature due to human activities, \
        primarily the emission of greenhouse gases (GHGs) like carbon dioxide (CO₂), methane (CH₄), and nitrous oxide (N₂O). \
        These gases trap heat in the Earth's atmosphere, creating a "greenhouse effect" that prevents heat from escaping \
        back into space. This causes the planet to warm up over time.

* **Why:** Global warming is primarily caused by human activities, particularly:
  1. Burning Fossil Fuels: The combustion of coal, oil, and natural gas for energy production \
  (such as electricity, heating, and transportation) releases large amounts of CO₂ into the atmosphere.
  2. Deforestation: Trees absorb CO₂ during photosynthesis. When forests are cleared for agriculture, \
  urbanization, or logging, this CO₂ is released, and fewer trees remain to absorb it.
  3. Industrial Processes: Some industrial activities release greenhouse gases such as methane \
  (from oil and gas extraction) and nitrous oxide (from agriculture and manufacturing).
  5. Waste Disposal: Landfills produce methane as organic waste decomposes anaerobically (without oxygen).

* **Pros:** While the overwhelming consensus is that global warming is detrimental, there are both potential positive and negative impacts:
  1. Longer Growing Seasons in Some Regions: Warmer temperatures may extend growing seasons in some colder \
  regions (e.g., parts of Canada, Russia), potentially increasing agricultural yields.
  2. New Shipping Routes: Melting Arctic ice could open up new shipping routes, reducing travel time for global trade.
  3. Access to Resources: Thawing ice could make natural resources, such as oil, gas, and minerals in \
  previously inaccessible regions, more available.

* **Cons:**
  1. Extreme Weather Events: Global warming leads to more intense and frequent heatwaves, droughts, floods, \
  hurricanes, and storms, resulting in significant human and economic costs.
  2. Sea Level Rise: Melting polar ice caps and glaciers cause sea levels to rise, threatening coastal communities, \
  ecosystems, and infrastructure.
  3. Food and Water Shortages: Altered precipitation patterns and droughts can reduce freshwater supplies, \
  impact agriculture, and lead to food insecurity.
"""
"\n\n"
"Question: {question}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are a helpful AI assistant, your name is Lili, designed by Dibyendu Biswas, an AI/ML Engineer.Your tasks is to answers the user\'s question in below following format:Question: What is Global Warming?Answer:\n* **What:** Global warming refers to the long-term increase in Earth\'s average surface temperature due to human activities,         primarily the emission of greenhouse gases (GHGs) like carbon dioxide (CO₂), methane (CH₄), and nitrous oxide (N₂O).         These gases trap heat in the Earth\'s atmosphere, creating a "greenhouse effect" that prevents heat from escaping         back into space. This causes the planet to warm up over time.\n  \n* **Why:** Global warming is primarily caused by human activities, particularly:\n  1. Burning Fossil Fuels: The com

In [79]:
chain2 = prompt | llm | StrOutputParser()

response = chain2.invoke("What is Data Warehouse?")
display(Markdown(response))

* **What:** A data warehouse is a central repository of integrated data from one or more disparate sources.  It stores current and historical data in one single place that are used for creating analytical reports for workers throughout the enterprise. Data warehouses are designed to support business intelligence (BI) activities, especially analytics.

* **Why:** Data warehouses are built to:
    1. **Consolidate Data:** Bring together data from multiple sources (e.g., operational databases, CRM systems, marketing platforms) into a single, consistent format.
    2. **Support Decision-Making:** Provide a platform for analyzing historical and current data to gain insights into business performance, trends, and customer behavior.
    3. **Improve Data Quality:** Cleanse, transform, and standardize data to ensure accuracy and consistency.
    4. **Enable Historical Analysis:** Track changes over time and analyze trends to understand how the business has evolved.
    5. **Facilitate Data Mining and Reporting:** Provide a rich source of data for generating reports, dashboards, and performing advanced analytics.

* **Pros:**
    1. **Improved Data Quality:** Data warehouses enforce data consistency and accuracy, leading to better decision-making.
    2. **Enhanced Business Intelligence:** They provide a comprehensive view of the business, enabling deeper insights and better strategic planning.
    3. **Increased Efficiency:** Data warehouses streamline data access and analysis, saving time and resources.
    4. **Historical Analysis:** They allow for trend analysis and understanding of past performance.
    5. **Better Decision Making:** Data-driven insights from warehouses empower businesses to make informed decisions.

* **Cons:**
    1. **Complexity:** Designing, implementing, and maintaining a data warehouse can be complex and time-consuming.
    2. **Cost:** Data warehouses can be expensive to build and maintain, requiring significant investment in hardware, software, and personnel.
    3. **Data Latency:** Data in a warehouse is not real-time; there is a delay between when data is generated and when it becomes available in the warehouse.
    4. **Data Inflexibility:**  Traditional data warehouses can be inflexible when it comes to adapting to changing business requirements or incorporating new data sources.
    5. **Over-reliance on IT:**  Business users may become overly reliant on IT to access and analyze data.


## **Iterative Prompting:**

**This is a process where you refine your prompt based on the outputs you get, slowly guiding the AI to the desired answer or style of answer**.<br>


Iterative Prompting is a technique where the **model is prompted multiple times in a loop, refining its output with each iteration**. <u>This approach is particularly useful for complex tasks that require progressive refinement or multi-step reasoning.</u>

## **Negative Prompting:**

In this method, **you tell the AI what not to do**. For instance, you might specify that you don’t want a certain type of content in the response.<br><br>

<u>Negative Prompting involves guiding the model to avoid certain undesired behaviors or responses.</u> This technique is useful for steering the model away from generating unwanted or harmful content (sex, politics, abuse, etc.).

In [82]:
from langchain.prompts import PromptTemplate

# Define the PromptTemplate with placeholders
prompt_template = """
Create a positive and ethical image description of a scene that emphasizes [positive aspects such as nature, creativity, joy, etc.].
Make sure the tone is respectful and suitable for all audiences.
The scene should avoid any harmful, controversial, sexual, political, or unethical themes.
Be sure to include these key elements: [specific objects, environment, or colors that fit the positive theme].
The overall mood should convey feelings of [positive emotion, such as happiness, peace, inspiration, etc.].
"""

# Instantiate the template with LangChain
template = PromptTemplate(
    input_variables=["positive_aspects", "key_elements", "emotion"],
    template=prompt_template
)

# Now you can use this template to fill in the placeholders
filled_prompt = template.format(
    positive_aspects="nature, tranquility, and harmony",
    key_elements="a serene mountain landscape, clear sky, and peaceful river",
    emotion="peace and awe"
)

# Print the generated prompt to verify the result
print(filled_prompt)



Create a positive and ethical image description of a scene that emphasizes [positive aspects such as nature, creativity, joy, etc.]. 
Make sure the tone is respectful and suitable for all audiences. 
The scene should avoid any harmful, controversial, sexual, political, or unethical themes.
Be sure to include these key elements: [specific objects, environment, or colors that fit the positive theme]. 
The overall mood should convey feelings of [positive emotion, such as happiness, peace, inspiration, etc.].



## **Hybrid Prompting:**


**Combining different prompting methods**, like **few-shot** with **chain-of-thought (CoT)** or **True-of-Thought (ToT)**, to get **more precise** or **creative** outputs.<br><br>

Hybrid Prompting combines multiple prompting techniques to handle complex tasks that require various strategies. This approach leverages the strengths of different methods to achieve better performance and flexibility.